$\newcommand{\giraffe}{\text{GiRaFFE}}$
# $\giraffe$: General Relativistic Force-Free Electrodynamics
## Porting the original $\giraffe$ code to NRPy+

Porting the original $\giraffe$ code to NRPy+ will generally make it easier to maintain, as well as to make changes. Specifically, it will make it nearly trivial to increase the finite-differencing order.

Our ultimate goal here will be to code the evolution equation 
\begin{equation}
\partial_t \tilde{S}_i = - \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) + \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu},
\end{equation}
where the densitized spatial Poynting flux one-form $\tilde{S}_i = \sqrt{\gamma} S_i$ (and $S_i$ comes from $S_{\mu} -n_{\nu} T^{\nu}_{{\rm EM} \mu}$) and
\begin{align}
T^{\mu \nu}_{\rm EM} &= b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu \nu} - b^{\mu} b^{\nu}, \\
\sqrt{4\pi} b^0 &= B^0_{\rm (u)} = \frac{u_j B^j}{\alpha}, \\
\sqrt{4\pi} b^i &= B^i_{\rm (u)} = \frac{B^i + (u_j B^j) u^i}{\alpha u^0}, \\
\end{align}
and 
$$B^i = \frac{\tilde{B}^i}{\gamma}.$$
Furthermore, the four-metric $g_{\mu\nu}$ is related to the three-metric $\gamma_{ij}$, lapse $\beta_i$, and shift $\alpha$ by
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$

Note that as usual, Greek indices refer to four-dimensional quantities where the zeroth component indicates $t$ components, while Latin indices refer to three-dimensional quantities. Since Python always indexes its lists from 0, however, the zeroth component will indicate a spatial direction, and any expressions involving mixed Greek and Latin indices will need to offset one set of indices by one.

## Preliminaries
First, we will import the core modules of NRPy that we will need. Then, we will declare the gridfunctions related to the metric and build the four metric using code from [Tutorial-smallb2_Poynting_vector-Cartesian.ipynb](Tutorial-smallb2_Poynting_vector-Cartesian.ipynb)

In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

thismodule = "GiRaFFE_NRPy"


Recall that the four-metric $g_{\mu\nu}$ is related to the three-metric $\gamma_{ij}$, lapse $\beta_i$, and shift $\alpha$ by
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$

In [2]:
# Step 2: Build the four metric
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01",DIM=3)
dummygammaUU, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
betaU   = ixp.register_gridfunctions_for_single_rank1("AUX","betaU",DIM=3)
alpha   = gri.register_gridfunctions("AUX","alpha")

# To get \gamma_{\mu \nu} = gammabar4DD[mu][nu], we'll need to construct the 4-metric, using Eq. 2.122 in B&S:
g4DD = ixp.zerorank2(DIM=4)

# Eq. 2.121 in B&S
betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] += gammaDD[i][j]*betaU[j]

# Now compute the beta contraction.
beta2 = sp.sympify(0)
for i in range(DIM):
    beta2 += betaU[i]*betaD[i]

# Eq. 2.122 in B&S
g4DD[0][0] = -alpha**2 + beta2
for mu in range(1,4):
    g4DD[mu][0] = g4DD[0][mu] = betaD[mu-1]
for mu in range(1,4):
    for nu in range(1,4):
        g4DD[mu][nu] = gammaDD[mu-1][nu-1]

We will also need spatial derivatives of the metric, $\partial_i g_{\mu\nu} = g_{\mu\nu,i}$. In terms of the three-metric, lapse, and shift, we find
$$
g_{\mu\nu,i} = \begin{pmatrix} 
-2\alpha \alpha_{,i} + \beta^k_{\ ,i} \beta_k + \beta^k \beta_{k,i} & \beta_{i,i} \\
\beta_{j,i} & \gamma_{ij,i}
\end{pmatrix}.
$$

Since this expression mixes Greek and Latin indices, we will need to store the expressions for each of the three spatial derivatives as separate variables. 
Also, consider the term $\beta_{i,j} = \partial_j \beta_i = \partial_j (\gamma_{ik} \beta^k) =  \gamma_{ik} \partial_j\beta^k + \beta^k \partial_j \gamma_{ik} = \gamma_{ik} \beta^k_{\ ,j} + \beta^k \gamma_{ik,j}$

In [3]:
# Step 3: Build the spatial derivative of the four metric
# Step 3a: Declare derivatives of grid functions. These will be handled by FD_outputC
alpha_dD   = ixp.declarerank1("alpha_dD")
betaU_dD   = ixp.declarerank2("betaU_dD","nosym")
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01")

# Step 3b: These derivatives will be constructed analytically.
betaDdD    = ixp.zerorank2()

# Zach says: Just set this up as a rank3 tensor with DIM=4. We'll not set the time deriv. 
#  This will get rid of a lot of unnecessary code below.
g4DDd0     = ixp.zerorank2(DIM=4)
g4DDd1     = ixp.zerorank2(DIM=4)
g4DDd2     = ixp.zerorank2(DIM=4)

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            betaDdD[i][j] = gammaDD[i][k] * betaU_dD[k][j] + betaU[k] * gammaDD_dD[i][k][j]

# Step 3c: Set the 00 components
g4DDd0[0][0] = -2*alpha*alpha_dD[0]
g4DDd1[0][0] = -2*alpha*alpha_dD[1]
g4DDd2[0][0] = -2*alpha*alpha_dD[2]
for k in range(DIM):
    g4DDd0[0][0] += betaU_dD[k][0] * betaD[k] + betaU[k] * betaDdD[k][0]
    g4DDd1[0][0] += betaU_dD[k][1] * betaD[k] + betaU[k] * betaDdD[k][1]
    g4DDd2[0][0] += betaU_dD[k][2] * betaD[k] + betaU[k] * betaDdD[k][2]
    
for mu in range(1,4):
    g4DDd0[mu][0] = g4DDd0[0][mu] = betaDdD[mu-1][0]
    g4DDd1[mu][0] = g4DDd0[0][mu] = betaDdD[mu-1][1]
    g4DDd2[mu][0] = g4DDd0[0][mu] = betaDdD[mu-1][2]
for mu in range(1,4):
    for nu in range(1,4):
        g4DDd0[mu][nu] = gammaDD_dD[mu-1][nu-1][0]
        g4DDd1[mu][nu] = gammaDD_dD[mu-1][nu-1][1]
        g4DDd2[mu][nu] = gammaDD_dD[mu-1][nu-1][2]
        

Now that the metric and its derivatives are out of the way, we will return our attention to the electromagnetic stress-energy tensor
$$T^{\mu \nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu \nu} - b^{\mu} b^{\nu}.$$
We will need the four-velocity $u^\mu$, where 
\begin{align}
u^i &= u^0 (\alpha v^i_{(n)} - \beta^i), \\
u_j &= \alpha u^0 \gamma_{ij} v^i_{(n)}, \\
\end{align}
and $v^i_{(n)}$ is the Valencia three-velocity. The values of $u^0$ and $v^i_{(n)}$ can be read from HydroBase and IllinoisGRMHD.


In [4]:
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU",DIM=3)

u0 = par.Cparameters("REAL",thismodule,"u0")
uD = ixp.zerorank1()

for i in range(DIM):
    for j in range(DIM):
        uD[j] += alpha*u0*gammaDD[i][j]*ValenciavU[j]


We also need the vector $b^{\mu}$ before we can compute this, which is related to the magnetic field by 
\begin{align}
b^0 &= \frac{1}{\sqrt{4\pi}} B^0_{\rm (u)} = \frac{u_j B^j}{\sqrt{4\pi}\alpha}, \\
b^i &= \frac{1}{\sqrt{4\pi}} B^i_{\rm (u)} = \frac{B^i + (u_j B^j) u^i}{\sqrt{4\pi}\alpha u^0}, \\
\end{align} and \begin{align}
B^i &= \frac{\tilde{B}^i}{\gamma},
\end{align}
where $\tilde{B}^i$ is the variable tracked by the HydroBase thorn in the Einstein Toolkit.

In [5]:
#M_PI = par.Cparameters("REAL",thismodule,"M_PI")
M_PI = sp.pi
BtildeU = ixp.register_gridfunctions_for_single_rank1("AUX","BtildeU",DIM=3)

BU = ixp.zerorank1()
for i in range(DIM):
    BU[i] = BtildeU[i]/gammaDET

# uBcontraction = u_i B^i
uBcontraction = sp.sympify(0)
for i in range(DIM):
    uBcontraction += uD[i]*BU[i]

# uU = 3-vector representing u^i = u^0 \left(\alpha v^i_{(n)} - \beta^i\right)
uU = ixp.zerorank1()
for i in range(DIM):
    uU[i] = u0*(alpha*ValenciavU[i] - betaU[i])

smallbU = ixp.zerorank1(DIM=4)
smallbU[0] = uBcontraction/(alpha*sp.sqrt(4*M_PI))
for i in range(DIM):
    smallbU[1+i] = (BU[i] + uBcontraction*uU[i])/(alpha*u0*sp.sqrt(4*M_PI))
    
smallbD = ixp.zerorank1(DIM=4)
for i in range(4):
    for j in range(4):
        smallbD[i] += g4DD[i][j]*smallbU[j]

smallb2 = sp.sympify(0)
for i in range(4):
    smallb2 += smallbU[i]*smallbD[i]


We now have all the pieces to calculate the stress-energy tensor,
$$T^{\mu \nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu \nu} - b^{\mu} b^{\nu}.$$


In [6]:
TEMUU = ixp.register_gridfunctions_for_single_rank2("AUX","TEMUU","sym01",DIM=4)

TEMUU[0][0] = smallb2*u0*u0 + smallb2*g4DD[0][0]/2 - smallbU[0]*smallbU[0]
for mu in range(1,4):
    TEMUU[mu][0] = TEMUU[0][mu] = smallb2*uU[mu-1]*u0 + smallb2*g4DD[mu][0]/2 - smallbU[mu]*smallbU[0]
for mu in range(1,4):
    for nu in range(1,4):
        TEMUU[mu][nu] = smallb2*uU[mu-1]*uU[nu-1] + smallb2*g4DD[mu][nu]/2 - smallbU[mu]*smallbU[nu]